In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import sqlite3
import seaborn as sns
from matplotlib.pyplot import figure
from bs4 import BeautifulSoup
import time
import requests     
import shutil       
import datetime
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')
import requests
import json
import xgboost
from xgboost import XGBClassifier
from random import randint
import  random
import os
import plotly.graph_objs as go
os.chdir('C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1')
from cmath import nan
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.tree import plot_tree
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.compose import make_column_selector as selector, ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
import pickle
from sklearn.metrics import fbeta_score
import winsound
from sklearn.linear_model import LinearRegression
from sklearn import tree, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, plot_confusion_matrix, recall_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve, auc, f1_score, make_scorer, recall_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

In [11]:
driver = webdriver.Chrome()
driver.minimize_window()

### Get Live Games Boxscores

In [30]:
nba_scoreboard = 'https://www.nba.com/games'
nba_scoreboard = requests.get(nba_scoreboard)
nba_scoreboard = BeautifulSoup(nba_scoreboard.content, 'html.parser')
# get all links
container = nba_scoreboard.find('div', class_ = 'GamesView_gameCardsContainer__c_9fB')
links = container.find_all('a')
# get hrefs for each link
hrefs = []
for link in links:
    hrefs.append(link.get('href'))
hrefs

# keep all boxscore links
boxscore_links = []
for href in hrefs:
    if 'box' in href:
        boxscore_links.append(href)
boxscore_links

boxes = pd.DataFrame(boxscore_links, columns = ['boxscore_link'])
boxes['boxscore_link'] = 'https://www.nba.com' + boxes['boxscore_link']
boxes



,boxscore_link
0,https://www.nba.com/game/gsw-vs-bos-0022200679...
1,https://www.nba.com/game/tor-vs-min-0022200680...
2,https://www.nba.com/game/chi-vs-det-0022200678...


In [13]:
def isBrowserAlive(driver):
   try:
      driver.current_url
      # or driver.title
      return True
   except:
      return False

In [51]:
if isBrowserAlive(driver) == False:
    driver = webdriver.Chrome()
    driver.minimize_window()
else:
    pass

In [35]:
def get_nba_boxscore(url):
    driver.get(url)
    time.sleep(5)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    # get container
    #container = soup.find('div', class_ = 'MaxWidthContainer_mwc__ID5AG')
    # get all tables
    tables = soup.find_all('table')
    # open first table
    t1 = pd.read_html(str(tables[0]))[0]
    t1['Player'] = t1['PLAYER'].apply(lambda x: x.split('.')[0][:-1])
    # min to str
    t1['MIN'] = t1['MIN'].astype(str)
    # drop rows if string of MIN includes DNP
    t1 = t1[~t1['MIN'].str.contains('DNP')]
    cols = ['Player', 'MIN', 'PTS', 'FGM', 'FGA', 'PF', 'REB', 'AST','STL', '+/-']
    t1 = t1[cols]

    t2 = pd.read_html(str(tables[1]))[0]
    t2['Player'] = t2['PLAYER'].apply(lambda x: x.split('.')[0][:-1])
    # min to str
    t2['MIN'] = t2['MIN'].astype(str)
    # drop rows if string of MIN includes DNP
    t2 = t2[~t2['MIN'].str.contains('DNP')]
    t2 = t2[cols]

    # append t1 and t2
    t1['Team'] = 'Home'
    t2['Team'] = 'Away'
    t3 = t1.append(t2)

    return t3

In [52]:
gsw = get_nba_boxscore(boxes['boxscore_link'][0])
my_players = ['Stephen Curry', 'Jordan Poole', 'Al Horford', 'Jayson Tatum', 'Draymond Green']
my_gsw = gsw[gsw['Player'].isin(my_players)]
my_gsw

,Player,MIN,PTS,FGM,FGA,PF,REB,AST,STL,+/-,Team
2,Draymond Green,23:23,4,1,3,2,11,6,0,2,Home
3,Jordan Poole,27:10,20,9,17,3,4,3,1,3,Home
4,Stephen Curry,29:40,22,7,16,0,3,7,3,17,Home
0,Jayson Tatum,29:56,21,5,17,2,12,5,2,-4,Away
1,Al Horford,24:28,14,6,9,2,9,0,0,-12,Away


In [53]:

tor = get_nba_boxscore(boxes['boxscore_link'][1])
my_players = ['Naz Reid', 'Fred VanVleet', 'Anthony Edwards']
my_tor = tor[tor['Player'].isin(my_players)]
my_tor

,Player,MIN,PTS,FGM,FGA,PF,REB,AST,STL,+/-,Team
4,Fred VanVleet,17:37,18,5,6,1,0,7,1,-2,Home
2,Naz Reid,03:54,0,0,2,3,1,0,0,1,Away
3,Anthony Edwards,17:54,9,3,11,2,1,3,1,-6,Away
